In [14]:
import numpy as np
from pysr import PySRRegressor

# 生成模拟数据
X = np.random.randn(100, 2)
x = X[:, 0]
v = X[:, 1]
X = np.column_stack((x, v))
y = 2 * x**2  + 3 * x * v + 5*v + np.cos(5*x)  # 添加噪声

# 配置 PySR
model = PySRRegressor(
    niterations=40,
    binary_operators=["+", "*", "-", "/"],
    unary_operators=["sin", "cos","log", "exp"],
    maxsize=20,
)

# 拟合模型
model.fit(X, y)

# 显示前5个最佳表达式
print("\n按损失值排序的前5个表达式：")
print("=" * 50)
sorted_equations = model.equations_.sort_values('loss', ascending=True)
for i in range(min(5, len(sorted_equations))):
    expr = sorted_equations.iloc[i]
    print(f"\n第{i+1}名表达式:")
    print(f"SymPy形式: {expr['sympy_format']}")
    print(f"损失值: {expr['loss']:.6e}")
    print("-" * 30)

c:\anacoda\envs\pytorch\Lib\site-packages\pysr\sr.py:2766: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:
[ Info: Final population:
[ Info: Results saved to:


c:\anacoda\envs\pytorch\Lib\site-packages\pysr\sr.py:2766: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:
[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           3.995e+01  1.594e+01  y = x₁
2           3.432e+01  1.518e-01  y = exp(x₁)
3           2.740e+01  2.255e-01  y = x₁ * 4.5437
4           2.502e+01  9.082e-02  y = exp(x₁ + x₀)
5           2.031e+01  2.085e-01  y = x₁ * (x₀ + 4.7108)
6           1.895e+01  6.923e-02  y = (exp(x₀) - -3.0799) * x₁
7           1.249e+01  4.167e-01  y = x₁ * ((x₀ - -1.4206) / 0.27585)
9           5.540e+00  4.066e-01  y = (x₀ + 1.2766) * ((x₁ / 0.2647) + x₀)
10          4.708e+00  1.627e-01  y = (((sin(x₀) + x₁) * x₀) + x₁) * 4.0609
11          1.312e+00  1.278e+00  y = ((x₀ * (x₀ + x₁)) + (x₁ * 2.107)) * 2.3907
13          4.171e-01  5.730e-01  y = (x₀ * ((x₀ * 2.0507) + (x₁ * 3.1708))) + (x₁ * 5.1542)
15          4.171e-01  5.994e-05  y = (((x₁ * 2.5095) + -0.004335) + (x₀ * (x₀ + (x₁ * 1.543...
                                      6)))) * 2.0539
17   

In [16]:
# 从specification文件中读取变量名和确定数学库
def get_var_mapping_and_math_lib(spec_file='specification_oscillator1_numpy.txt'):
    """从specification文件中读取变量名和确定数学库"""
    with open(spec_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 找到equation函数的参数定义
    import re
    match = re.search(r'def equation\((.*?)\)', content)
    if not match:
        raise ValueError("无法在specification文件中找到equation函数定义")
    
    # 提取参数及其类型
    param_defs = [p.strip() for p in match.group(1).split(',') if 'params' not in p]
    if not param_defs:
        raise ValueError("未找到有效的变量参数")

    # 创建变量映射和收集所有类型
    var_mapping = {}
    type_modules = set()
    
    for i, param in enumerate(param_defs):
        name_type = param.split(':')
        if len(name_type) != 2:
            continue
        
        name, type_hint = name_type
        name = name.strip()
        var_mapping[f'x{i}'] = name
        
        # 提取模块名（np, jnp, torch等）
        type_match = re.search(r'(\w+)\.', type_hint)
        if type_match:
            type_modules.add(type_match.group(1))
    
    # 根据类型优先级选择数学库前缀
    # 优先使用非numpy的库（因为它们可能有特殊的需求）
    if 'torch' in type_modules:
        math_prefix = 'torch.'
    elif 'jnp' in type_modules:
        math_prefix = 'jnp.'
    else:
        math_prefix = 'np.'
    
    return var_mapping, math_prefix

def replace_math_functions(expr: str, math_prefix: str) -> str:
    """替换数学函数为指定库的版本"""
    import re
    # 定义需要替换的数学函数
    math_funcs = ['sin', 'cos', 'log', 'exp']
    
    # 替换所有未带前缀的数学函数
    for func in math_funcs:
        # 使用原始字符串(r'')来避免转义序列问题
        pattern = r'(?<!\.)(?<!\w)' + func + r'(?!\w)'
        expr = re.sub(pattern, f'{math_prefix}{func}', expr)
    
    return expr

def replace_constants_with_params(expr: str) -> str:
    """将表达式中的常数替换为params数组项，包括处理除法形式"""
    import re
    param_counter = 0
    
    def replace_with_param(match):
        nonlocal param_counter
        number = match.group(0)
        current = param_counter
        param_counter += 1
        if number.startswith('/'):
            return f"*params[{current}]"
        else:
            return f"params[{current}]"

    # 1. 首先匹配除法形式的数字
    expr = re.sub(r'/(-?\d+\.?\d*)', replace_with_param, expr)
    
    # 2. 然后匹配其他数字（排除科学记数法中的指数）
    expr = re.sub(r'(?<![e|E])(?<!/)-?\d+\.?\d*(?![e|E]\d+)', replace_with_param, expr)
    
    return expr

# 处理文件内容
def clean_and_prepare_content(file_path):
    """读取文件，删除已存在的表达式块，并准备新内容"""
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 分割内容以找到所有的 INITIAL BODIES 块
    parts = content.split("# === INITIAL BODIES START ===")
    
    if len(parts) > 1:
        # 保留第一部分（主要内容）
        base_content = parts[0].rstrip()
    else:
        base_content = content.rstrip()
        
    return base_content

try:
    # 获取变量映射和数学库前缀
    var_mapping, math_prefix = get_var_mapping_and_math_lib()
    print(f"从specification文件中提取的变量映射: {var_mapping}")
    print(f"使用的数学库前缀: {math_prefix}")
    
    # 处理表达式并准备添加到文件
    processed_expressions = []
    for i in range(min(5, len(sorted_equations))):
        expr = sorted_equations.iloc[i]
        sympy_expr = str(expr['sympy_format'])
        
        # 1. 替换变量名
        for old_var, new_var in var_mapping.items():
            sympy_expr = sympy_expr.replace(old_var, new_var)
        
        # 2. 替换数学函数
        sympy_expr = replace_math_functions(sympy_expr, math_prefix)
        
        # 3. 替换常数为params数组项
        sympy_expr = replace_constants_with_params(sympy_expr)
        
        # 构建完整的函数体
        body = f"dv = {sympy_expr}\nreturn dv"
        processed_expressions.append(body)
        
        # 打印处理后的表达式
        print(f"\n第{i+1}名表达式:")
        print(f"处理后的形式:\n{body}")
        print("-" * 30)
    
    # 清理并准备文件内容
    file_path = 'specification_oscillator1_numpy.txt'
    base_content = clean_and_prepare_content(file_path)
    
    # 构建新内容
    new_content = [
        base_content,  # 原始内容
        "\n",  # 空行
        "# === INITIAL BODIES START ===" 
    ]
    
    # 添加每个表达式
    for expr in processed_expressions:
        new_content.extend([
            "",
            "# --- BODY START ---",
            expr,
            "# --- BODY END ---"
        ])
    
    # 添加结束标记
    new_content.extend([
        "",
        "# === INITIAL BODIES END ==="
    ])
    
    # 写回文件
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(new_content))
    
    print("\n处理完成！已将表达式添加到文件，并确保没有重复内容")

except Exception as e:
    print(f"错误: {str(e)}")

从specification文件中提取的变量映射: {'x0': 'x', 'x1': 'v'}
使用的数学库前缀: torch.

第1名表达式:
处理后的形式:
dv = (x*(x + v*params[params[1]] + torch.sin(torch.exp(x))*(params[2])) + v*params[3])*params[4]
return dv
------------------------------

第2名表达式:
处理后的形式:
dv = (x*(x + v*params[0] + torch.sin(v)*(params[1])) + v*params[2])*params[3]
return dv
------------------------------

第3名表达式:
处理后的形式:
dv = (v*params[0] + (x + v*(params[1]))*(x + v*params[2]))*params[3]
return dv
------------------------------

第4名表达式:
处理后的形式:
dv = (x*(x + v*params[0]) + v*params[1] - params[2])*params[3]
return dv
------------------------------

第5名表达式:
处理后的形式:
dv = x*(x*params[0] + v*params[1]) + v*params[2]
return dv
------------------------------

处理完成！已将表达式添加到文件，并确保没有重复内容
